In [ ]:
from gurobipy import *
from IPython.display import Image
from scipy.stats import poisson, nbinom
import matplotlib.pyplot as plt

import random
import pandas as pd
import numpy as np
import math
import os
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
Image("objective_function.png")

In [ ]:
raw_df = pd.DataFrame()
files = os.listdir('data')
for f in files:
    if 'item' not in f:
        df = pd.read_csv('data/{}'.format(f))
        raw_df = pd.concat([raw_df, df])

In [ ]:
# raw_df.info()
# raw_df.head(10)

In [ ]:
# onpromotion沒有完整一年的資料
promo_df = raw_df.loc[~raw_df['onpromotion'].isna()]
list(promo_df.loc[(promo_df['date']>='2014-01-01') & (promo_df['date']<='2014-12-31')].groupby('item_nbr').size()).index(365)


In [ ]:
# 每個品項都有完整兩年的資料
for i in list(raw_df.loc[(raw_df['date']>='2014-01-01') & (raw_df['date']<='2015-12-31')].groupby('item_nbr').size()):
    if i != 730:
        print(i)

In [ ]:
integer_items = []
item_dfs = list(raw_df.groupby('item_nbr'))

for item_df in item_dfs:
    stop = False
    for us in item_df[1]['unit_sales']:
        if (us.is_integer() != True) or (us < 0):
            stop = True
            break
    if stop:
        continue
    # print(item_df[0])
    integer_items.append(item_df[0])
# integer_items[:10]
    

In [ ]:
# 函式庫
def get_decisions(day, model):

    # 用測試資料計算CDF
    F = {}
    for i in N:
        for d in list(range(U[i]+1)):
            item_nbr = integer_items[i]
            if model == 'poisson':
                F[i, d] = poisson.cdf(k = d, 
                                    mu = poisson_pred_df['pred'].loc[(poisson_pred_df['item_nbr']==item_nbr) & (poisson_pred_df['day']==day)].values[0])
            elif model == 'nb2':
                # np.MachAr().eps
                mean = nb2_pred_df['pred'].loc[(nb2_pred_df['item_nbr']==item_nbr) & (nb2_pred_df['day']==day)].values[0]
                alpha = max(nb2_pred_df['alpha'].loc[(nb2_pred_df['item_nbr']==item_nbr) & (nb2_pred_df['day']==day)].values[0], np.MachAr().eps)
                variance = mean * (1 + alpha*mean)
                
                F[i, d] = nbinom.cdf(k = d, 
                                    n = mean**2 / (variance - mean), 
                                    p = mean / variance)
            elif model == 'gp':
                F[i, d] = gp_cdf(K = d, 
                                mu = gp_pred_df['pred'].loc[(gp_pred_df['item_nbr']==item_nbr) & (gp_pred_df['day']==day)].values[0],
                                alpha = alpha_list[i])
            
    #             print(i, d)
    #             print('mean:', mean)
    #             print('alpha:', alpha)
    #             print('variance:', variance)
    #             print('n:', (mean**2 / (variance - mean)))
    #             print('p:', mean / variance)
    #             print('-----------------')
    # print(F)

    # 建立模型
    m = Model("freshfood_newsvendor")
    
    # 不顯示結果
    m.setParam('OutputFlag', 0)
    
    # 決策變數
    q = m.addVars(N, vtype = GRB.INTEGER, name='q')
    z_varnames = []
    for i in N:
        for d in list(range(L[i]+1, U[i]+1)):
            z_varnames.append((i, d))
    z = m.addVars(z_varnames, vtype = GRB.BINARY, name='z')

    # 更新模型
    m.update()

    # quicksum()等於加總符號sigma
    # 目標函式
    m.setObjective(quicksum((p[i] - c[i]) * q[i] - 
                (p[i] * quicksum(F[i, d] for d in range(L[i])) + 
                    quicksum(p[i] * F[i, d] * z[i, d+1] for d in range(L[i], U[i]))) for i in N), ## L[i]+1 變成 L[i], U[i] 變成 q[i]
                sense = GRB.MAXIMIZE)

    # 限制式
    m.addConstrs(q[i] >= L[i] for i in N)
    m.addConstrs(q[i] <= U[i] for i in N)

    m.addConstrs(quicksum(r[i, j] * q[i] for i in N) <= R[j]  for j in J)

    m.addConstrs(q[i] == L[i] + quicksum(z[i, d] for d in range(L[i]+1, U[i]+1)) for i in N)

    m.addConstrs(z[i, d-1] >= z[i, d] for i in N
                                    for d in range(L[i]+2, U[i]+1))

    # 開始最佳化運算
    m.optimize()

    # 印出模型 
    # print (m.display())
    # m.write('mip1.lp')

    # for v in m.getVars():
    #     print('%s %g' %(v.varName, v.x))

    # print('Obj: %g' % m.objVal)

    # 每個品項的最佳訂購量
    decisions = []
    for i in N:
        decisions.append([var for var in m.getVars() if "q[{}]".format(i) in var.VarName][0].X)


    return decisions

def get_poisson_mu(item_nbr, from_date, to_date, test_date):
    
    # 訓練集
    train_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=to_date)]
    X_train = train_data.iloc[:, 8:]
    y_train = train_data[['unit_sales']]

    # 測試集
    test_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>to_date) & (df['date']<=test_date)]
    X_test = test_data.iloc[:, 8:]
    y_test = test_data[['unit_sales']]

    # 訓練poisson regression模型
    poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
    pred = poisson_model.predict(X_test)
    
    # 將預測結果放入poisson_pred_df
    result_df = pd.DataFrame(pred, columns=['pred'])
    result_df['item_nbr'] = item_nbr
    result_df = result_df.reset_index(drop=True).reset_index()
    result_df.rename(columns={'index':'day'}, inplace=True)

    global poisson_pred_df
    poisson_pred_df = pd.concat([poisson_pred_df, result_df], axis=0)
    
    # get mu for nb2 model
    temp_df = pd.DataFrame(poisson_model.mu, columns=['mu'])
    temp_df['item_nbr'] = item_nbr
    temp_df = temp_df.reset_index(drop=True).reset_index()
    temp_df.rename(columns={'index':'day'}, inplace=True)

    global mu_df
    mu_df = pd.concat([mu_df, temp_df], axis=0)

def get_nb2_mu(item_nbr, from_date, to_date, test_date):
    
    # train auxiliary OLS regression model to get alpha
    train_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=to_date)]
    train_data.reset_index(drop=True, inplace=True)
    train_data['LAMBDA'] = mu_df['mu'].loc[mu_df['item_nbr']==item_nbr]
    train_data['AUX_OLS_DEP'] = train_data.apply(lambda x: ((x['unit_sales'] - x['LAMBDA'])**2 - x['LAMBDA']) / x['LAMBDA'], axis=1)
    
    # aux_olsr_model.params[0]即為alpha
    ols_expr = """AUX_OLS_DEP ~ LAMBDA - 1"""
    aux_olsr_model = smf.ols(ols_expr, train_data).fit()

    # 訓練集
    train_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=to_date)]
    X_train = train_data.iloc[:, 8:]
    y_train = train_data[['unit_sales']]

    # 測試集
    test_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>to_date) & (df['date']<=test_date)]
    X_test = test_data.iloc[:, 8:]
    y_test = test_data[['unit_sales']]

    # 訓練NB2模型
    nb2_model = sm.GLM(y_train, X_train, family=sm.families.NegativeBinomial(alpha=aux_olsr_model.params[0])).fit()
    pred = nb2_model.predict(X_test)

    # 將預測結果放入nb2_pred_df
    result_df = pd.DataFrame(pred, columns=['pred'])
    result_df['item_nbr'] = item_nbr
    result_df['alpha'] = aux_olsr_model.params[0]
    result_df = result_df.reset_index(drop=True).reset_index()
    result_df.rename(columns={'index':'day'}, inplace=True)

    global nb2_pred_df
    nb2_pred_df = pd.concat([nb2_pred_df, result_df], axis=0)

def get_gp_mu(p, item_nbr, from_date, to_date, test_date):
    
    # 訓練集
    train_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=to_date)]
    X_train = train_data.iloc[:, 8:]
    y_train = train_data[['unit_sales']]

    # 測試集
    test_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>to_date) & (df['date']<=test_date)]
    X_test = test_data.iloc[:, 8:]
    y_test = test_data[['unit_sales']]

    # 訓練generalized poisson regression模型
    gp_model = sm.GeneralizedPoisson(y_train, X_train, p=p).fit()
    pred = gp_model.predict(X_test)
    
    # 將預測結果放入gp_pred_df
    result_df = pd.DataFrame(gp_model.predict(X_test), columns=['pred'])
    result_df['item_nbr'] = item_nbr
    result_df = result_df.reset_index(drop=True).reset_index()
    result_df.rename(columns={'index':'day'}, inplace=True)

    global gp_pred_df
    gp_pred_df = pd.concat([gp_pred_df, result_df], axis=0)

    # 得出alpha
    global alpha_list
    alpha_list.append(gp_model.params['alpha'])
    print(gp_model.params['alpha'])

def cal_mse(item_nbr, model):
    # 查看模型的MSE
    y_true = actual_sales['unit_sales'].loc[actual_sales['item_nbr']==item_nbr]
    if model == 'poisson':
        y_pred = poisson_pred_df['pred'].loc[poisson_pred_df['item_nbr']==item_nbr]
    elif model == 'nb2':
        y_pred = nb2_pred_df['pred'].loc[(nb2_pred_df['item_nbr']==item_nbr)]
    elif model == 'gp':
        y_pred = gp_pred_df['pred'].loc[(gp_pred_df['item_nbr']==item_nbr)]
    # print(y_true)
    # print(y_pred)
    return mean_squared_error(y_true, y_pred)

def gp_cdf(K, mu, alpha):
    cdf = 0
    for k in range(K+1):
        cdf += (mu * ((mu + alpha * k)** (k-1)) * math.exp(-mu - alpha * k)) / math.factorial(k)
    
    return cdf


In [ ]:
day = 0
F = {}
for i in N:
    for d in list(range(U[i]+1)):
        item_nbr = integer_items[i]
        F[i, d] = gp_cdf(K = d, 
                        mu = gp_pred_df['pred'].loc[(gp_pred_df['item_nbr']==item_nbr) & (gp_pred_df['day']==day)].values[0],
                        alpha = alpha_list[i])
F

In [ ]:
# pd.set_option('display.max_rows', None)

# 定義變數們
# 用2014年的資料當訓練集，並用後180天的資料當測試集
from_date = '2014-01-01'
to_date = '2014-12-31'
test_day = 180
test_date = (datetime.datetime.strptime(to_date, "%Y-%m-%d") + datetime.timedelta(days=test_day)).strftime("%Y-%m-%d")

raw_df['month'] = raw_df['date'].apply(lambda d: datetime.datetime.strptime(d, '%Y-%m-%d').month)
df = pd.get_dummies(raw_df, columns=['weekday', 'month', 'type'], drop_first=True)

poisson_pred_df = pd.DataFrame()
nb2_pred_df = pd.DataFrame()
gp_pred_df = pd.DataFrame()
alpha_list = []
mu_df = pd.DataFrame()

p = [30, 50, 60, 70, 50, 40, 20, 60, 80, 100]
c = [10, 26, 37, 25, 20, 15, 10, 31, 31, 40]

# L：下限
L = []
for item_nbr in integer_items[:10]:
    all_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=test_date)]
    L.append(int(np.percentile(all_data['unit_sales'], 5)))

# U：上限
U = []
for item_nbr in integer_items[:10]:
    all_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=test_date)]
    U.append(int(np.percentile(all_data['unit_sales'], 95)))

R = [sum(U)]
J = list(range(1))
N = list(range(7))

# r：每個品項所占資源單位
r = {}
for i in N:
    for j in J:
        r[i, j] = random.randint(1, 1)

# 若unit_sales > U 即為 U
# for i in N:
#     df['unit_sales'].loc[df['item_nbr']==integer_items[i]] = df['unit_sales'].loc[df['item_nbr']==integer_items[i]].apply(lambda x: x if x <= U[i] else U[i])
# print(df)


In [ ]:
## 去掉q的版本
def get_decisions(day, model):

    # 用測試資料計算CDF
    F = {}
    for i in N:
        for d in list(range(U[i]+1)):
            item_nbr = integer_items[i]
            if model == 'poisson':
                F[i, d] = poisson.cdf(k = d, 
                                    mu = poisson_pred_df['pred'].loc[(poisson_pred_df['item_nbr']==item_nbr) & (poisson_pred_df['day']==day)].values[0])
            elif model == 'nb2':
                # np.MachAr().eps
                mean = nb2_pred_df['pred'].loc[(nb2_pred_df['item_nbr']==item_nbr) & (nb2_pred_df['day']==day)].values[0]
                alpha = max(nb2_pred_df['alpha'].loc[(nb2_pred_df['item_nbr']==item_nbr) & (nb2_pred_df['day']==day)].values[0], np.MachAr().eps)
                variance = mean * (1 + alpha*mean)
                
                F[i, d] = nbinom.cdf(k = d, 
                                    n = mean**2 / (variance - mean), 
                                    p = mean / variance)
    #             print(i, d)
    #             print('mean:', mean)
    #             print('alpha:', alpha)
    #             print('variance:', variance)
    #             print('n:', (mean**2 / (variance - mean)))
    #             print('p:', mean / variance)
    #             print('-----------------')
    # print(F)

    # 建立模型
    m = Model("freshfood_newsvendor")
    
    # 不顯示結果
    # m.setParam('OutputFlag', 0)
    
    # 決策變數
    z_varnames = []
    for i in N:
        for d in list(range(L[i], U[i]+1)): # L[i]+1 變成 L[i]
            z_varnames.append((i, d))
    z = m.addVars(z_varnames, vtype = GRB.BINARY, name='z')

    # 更新模型
    m.update()

    # quicksum()等於加總符號sigma
    # 目標函式
    m.setObjective(quicksum((p[i] - c[i]) * (L[i] + quicksum(z[i, d] for d in range(L[i]+1, U[i]+1)))- 
                (p[i] * quicksum(F[i, d] for d in range(L[i])) + 
                    quicksum(p[i] * F[i, d] * z[i, d] for d in range(L[i], L[i] + quicksum(z[i, d] for d in range(L[i]+1, U[i]+1))-1))) for i in N), ## L[i]+1 變成 L[i], U[i] 變成 q[i]
                sense = GRB.MAXIMIZE)

    # 限制式(印出來)
    m.addConstrs(L[i] + quicksum(z[i, d] for d in range(L[i]+1, U[i]+1)) >= L[i] for i in N)
    m.addConstrs(L[i] + quicksum(z[i, d] for d in range(L[i]+1, U[i]+1)) <= U[i] for i in N)

    m.addConstrs(quicksum(r[i, j] * q[i] for i in N) <= R[j]  for j in J)

    m.addConstrs(z[i, d-1] >= z[i, d] for i in N
                                    for d in range(L[i]+2, U[i]+1))

    # 開始最佳化運算
    m.optimize()

    # 印出模型 
    print (m.display())
    # m.write('mip1.lp')

    for v in m.getVars():
        print('%s %g' %(v.varName, v.x))

    print('Obj: %g' % m.objVal)

    # 每個品項的最佳訂購量
    decisions = []
    for i in N:
        decisions.append([var for var in m.getVars() if "q[{}]".format(i) in var.VarName][0].X)


    return decisions

In [ ]:
# 得出每個品項在未來180天的預測值
for item_nbr in integer_items[:10]:
    get_poisson_mu(item_nbr, from_date, to_date, test_date)
    get_nb2_mu(item_nbr, from_date, to_date, test_date)
    get_gp_mu(1, item_nbr, from_date, to_date, test_date)

In [ ]:
# 真實銷售數據
actual_sales = pd.DataFrame()
for item_nbr in integer_items[:10]:
    test_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>to_date) & (df['date']<=test_date)]
    y_test = pd.DataFrame(test_data[['unit_sales']])
    y_test['item_nbr'] = item_nbr
    y_test = y_test.reset_index(drop=True).reset_index()
    y_test.rename(columns={'index':'day'}, inplace=True)

    actual_sales = pd.concat([actual_sales, y_test], axis=0)

In [ ]:
# 查看模型的MSE
for i in range(10):
    print(cal_mse(integer_items[i], 'gp'))

In [ ]:
# actual_sales['unit_sales'].loc[actual_sales['item_nbr']==integer_items[0]].describe()
# plt.plot(actual_sales['unit_sales'].loc[actual_sales['item_nbr']==integer_items[0]])

## 進行最佳化

In [ ]:
# test_day = 1
poisson_decisions = []

for day in range(test_day):
    # print(day)
    poisson_decisions.append(get_decisions(day, 'poisson'))
# poisson_decisions

In [ ]:
poisson_decisions

In [ ]:
# gp預測值有空值
# test_day = 1
gp_decisions = []

for day in range(test_day):
    # print(day)
    gp_decisions.append(get_decisions(day, 'gp'))
# poisson_decisions

In [ ]:
## alpha為負值時，p會大於1，n會是負值，導致cdf為nan
## mean、variance太小時，n會無限大
## 要印出產品銷售統計資料

# test_day = 180
nb2_decisions = []

for day in range(test_day):
    # print(day)
    nb2_decisions.append(get_decisions(day, 'nb2'))
#nb2_decisions    

### Poisson

In [ ]:
# Poisson 訂購max(min(E(y|x), U), L)
poisson_profits = []
poisson_lostsales = pd.DataFrame()
for day in range(test_day):
    day_profit = []
    day_lostsale = []
    for i in N:
        order = round(max(min(poisson_pred_df['pred'].loc[(poisson_pred_df['item_nbr']==integer_items[i]) & (poisson_pred_df['day']==day)].values[0], U[i]), L[i]))
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append(p[i] * min(actual_sale, order) - c[i] * order)

        day_lostsale.append([day, integer_items[i], p[i], order, actual_sale, max(actual_sale - order, 0)])
        
    poisson_profits.append(day_profit)
    day_lostsale = pd.DataFrame(day_lostsale, columns=['day', 'item_nbr', 'price', 'order', 'actual_sale', 'lost_sales'])
    poisson_lostsales = pd.concat([poisson_lostsales, day_lostsale])

poisson_lostsales['lost_sales*price'] = poisson_lostsales['price'] * poisson_lostsales['lost_sales']
# poisson_lostsales

In [ ]:
# for day in range(test_day):
    # print('第{}天的利潤: {}'.format(day, poisson_profits[day]))
total_poisson_profit = sum(sum(profit) for profit in poisson_profits)
print('****總利潤: {}****'.format(total_poisson_profit))

print('****總銷售損失(數量): {}****'.format(sum(poisson_lostsales['lost_sales'])))
print('****總銷售損失(數量*售價): {}****'.format(sum(poisson_lostsales['lost_sales*price'])))


In [ ]:
# Poisson最佳訂購量
poisson_opti_profits = []
poisson_opti_lostsales = pd.DataFrame()

for day in range(test_day):
    day_profit = []
    day_opti_lostsale = []
    for i in N:
        order = poisson_decisions[day][i]
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append(p[i] * min(actual_sale, order) - c[i] * order)
        
        day_lostsale.append([day, integer_items[i], p[i], order, actual_sale, max(actual_sale - order, 0)])
        
    poisson_opti_profits.append(day_profit)
    day_lostsale = pd.DataFrame(day_lostsale, columns=['day', 'item_nbr', 'price', 'order', 'actual_sale', 'lost_sales'])
    poisson_opti_lostsales = pd.concat([poisson_opti_lostsales, day_lostsale])

poisson_opti_lostsales['lost_sales*price'] = poisson_opti_lostsales['price'] * poisson_opti_lostsales['lost_sales']
# poisson_opti_lostsales

In [ ]:
# for day in range(test_day):
#     print('第{}天的利潤: {}'.format(day, poisson_opti_profits[day]))
total_poisson_opti_profit = sum(sum(profit) for profit in poisson_opti_profits)
print('****總利潤: {}****'.format(total_poisson_opti_profit))

print('****總銷售損失(數量): {}****'.format(sum(poisson_opti_lostsales['lost_sales'])))
print('****總銷售損失(數量*售價): {}****'.format(sum(poisson_opti_lostsales['lost_sales*price'])))

### GP1

In [ ]:
# GP1 訂購max(min(E(y|x), U), L)
gp_profits = []
gp_lostsales = pd.DataFrame()

for day in range(test_day):
    
    day_profit = []
    day_lostsale = []
    for i in N:
        order = round(max(min(gp_pred_df['pred'].loc[(gp_pred_df['item_nbr']==integer_items[i]) & (gp_pred_df['day']==day)].values[0], U[i]), L[i]))
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append(p[i] * min(actual_sale, order) - c[i] * order)
        
        day_lostsale.append([day, integer_items[i], p[i], order, actual_sale, max(actual_sale - order, 0)])
        
    gp_profits.append(day_profit)
    day_lostsale = pd.DataFrame(day_lostsale, columns=['day', 'item_nbr', 'price', 'order', 'actual_sale', 'lost_sales'])
    gp_lostsales = pd.concat([gp_lostsales, day_lostsale])

gp_lostsales['lost_sales*price'] = gp_lostsales['price'] * gp_lostsales['lost_sales']
# gp_lostsales

In [ ]:
# for day in range(test_day):
    # print('第{}天的利潤: {}'.format(day, gp_profits[day]))
total_gp_profit = sum(sum(profit) for profit in gp_profits)
print('****總利潤: {}****'.format(total_gp_profit))

print('****總銷售損失(數量): {}****'.format(sum(gp_lostsales['lost_sales'])))
print('****總銷售損失(數量*售價): {}****'.format(sum(gp_lostsales['lost_sales*price'])))


In [ ]:
# GP1最佳訂購量
gp_opti_profits = []
gp_opti_lostsales = pd.DataFrame()

for day in range(test_day):
    
    day_profit = []
    day_opti_lostsale = []
    for i in N:
        order = gp_decisions[day][i]
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append(p[i] * min(actual_sale, order) - c[i] * order)
        
        day_opti_lostsale.append([day, integer_items[i], p[i], order, actual_sale, max(actual_sale - order, 0)])
        
    gp_opti_profits.append(day_profit)
    day_opti_lostsale = pd.DataFrame(day_opti_lostsale, columns=['day', 'item_nbr', 'price', 'order', 'actual_sale', 'lost_sales'])
    gp_opti_lostsales = pd.concat([gp_opti_lostsales, day_opti_lostsale])

gp_opti_lostsales['lost_sales*price'] = gp_opti_lostsales['price'] * gp_opti_lostsales['lost_sales']
# gp_opti_lostsales

In [ ]:
# for day in range(test_day):
#     print('第{}天的利潤: {}'.format(day, gp_opti_profits[day]))
total_gp_opti_profit = sum(sum(profit) for profit in gp_opti_profits)
print('****總利潤: {}****'.format(total_gp_opti_profit))

print('****總銷售損失(數量): {}****'.format(sum(gp_opti_lostsales['lost_sales'])))
print('****總銷售損失(數量*售價): {}****'.format(sum(gp_opti_lostsales['lost_sales*price'])))

### NB2

In [ ]:
# NB2 訂購max(min(E(y|x), U), L)
nb2_profits = []
for day in range(test_day):
    day_profit = []
    for i in N:
        order = round(max(min(nb2_pred_df['pred'].loc[(nb2_pred_df['item_nbr']==integer_items[i]) & (nb2_pred_df['day']==day)].values[0], U[i]), L[i]))
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append(p[i] * min(actual_sale, order) - c[i] * order)
    
    nb2_profits.append(day_profit)

In [ ]:
# for day in range(test_day):
    # print('第{}天的利潤: {}'.format(day, nb2_profits[day]))
total_nb2_profit = sum(sum(profit) for profit in nb2_profits)
print('****總利潤: {}****'.format(total_nb2_profit))

In [ ]:
# NB2最佳訂購量
nb2_opti_profits = []
for day in range(test_day):
    day_profit = []
    for i in N:
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append(p[i] * min(actual_sale, nb2_decisions[day][i]) - c[i] * nb2_decisions[day][i])
        
    nb2_opti_profits.append(day_profit)

In [ ]:
# for day in range(test_day):
#     print('第{}天的利潤: {}'.format(day, nb2_opti_profits[day]))
total_nb2_opti_profit = sum(sum(profit) for profit in nb2_opti_profits)
print('****總利潤: {}****'.format(total_nb2_opti_profit))

In [ ]:
# 訂購平均數量的總利潤
equal_profits = []
for day in range(test_day):
    day_profit = []
    for i in N:
        # print(int(R[0]/len(N)))
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append(p[i] * min(actual_sale, R[0]/len(N)) - c[i] * int(R[0]/len(N)))
    
    equal_profits.append(day_profit)

In [ ]:
# for day in range(test_day):
#     print('第{}天的利潤: {}'.format(day, equal_profits[day]))
total_equal_profit = sum(sum(profit) for profit in equal_profits)
print('****總利潤: {}****'.format(total_equal_profit))

### Optimal_profit

In [ ]:
# optimal_profit：每天訂購真實需求量
optimal_profits = []
for day in range(test_day):
    day_profit = []
    for i in N:
        actual_sale = actual_sales['unit_sales'].loc[(actual_sales['item_nbr']==integer_items[i]) & (actual_sales['day']==day)].values[0]
        day_profit.append((p[i] * actual_sale) - (c[i] * actual_sale))
        
    optimal_profits.append(day_profit)

In [ ]:
# for day in range(test_day):
#     print('第{}天的利潤: {}'.format(day, opti_profits[day]))
total_optimal_profit = sum(sum(profit) for profit in optimal_profits)
print('****總利潤: {}****'.format(total_optimal_profit))

In [ ]:
# 計算模型gap：(optimal_profit - model_profit) / optimal_profit
# poisson_model
print("只用poisson:", (total_optimal_profit - total_poisson_profit) / total_optimal_profit)

# gp_model
print("只用gp:", (total_optimal_profit - total_gp_profit) / total_optimal_profit)

# poisson_opti_model
print("poisson最佳化:", (total_optimal_profit - total_poisson_opti_profit) / total_optimal_profit)

# gp_opti_model
print("gp最佳化:", (total_optimal_profit - total_gp_opti_profit) / total_optimal_profit)


### 代辦事項

In [2]:
## 模型背景、做的東西(數學式跟程式的差異 poisson)、例子寫成文件
## 整理程式(7/16)
## mdn
## 開會時間六日上午
## 所有p*2 看lostsales的差異
## github連結
## 工研院進用問題

In [ ]:
print("L:", L)
print("U:", U)
print("R:", R)

In [ ]:
p = [30, 50, 60, 70, 50, 40, 20, 60, 80, 100]
c = [10, 26, 37, 25, 20, 15, 10, 31, 31, 40]
day = 0
F = {}
for i in N:
    for d in list(range(U[i]+1)):
        item_nbr = integer_items[i]
        F[i, d] = poisson.cdf(k = d, 
                                mu = poisson_pred_df['pred'].loc[(poisson_pred_df['item_nbr']==item_nbr) & (poisson_pred_df['day']==day)].values[0])
F

In [ ]:
p[5]*F[5, 0] + p[6]*F[6,0] + p[6]*F[6,1] + p[6]*F[6,2] + p[6]*F[6,3]

In [ ]:
# 驗算的地方
p = 3
q = 7
L = 5
U = 11
day = 1

F = {}
for i in N:
    for d in list(range(U+1)):
        item_nbr = integer_items[i]
        F[i, d] = poisson.cdf(k=d, mu=poisson_pred_df['pred'].loc[(poisson_pred_df['item_nbr']==item_nbr) & (poisson_pred_df['day']==day)].values[0])

z = []
for i in range(L+1, U+1):
    if i <= q:
        z.append(1)
    else:
        z.append(0)
z

In [ ]:
rule1 = 0
for d in range(0, q):
    rule1 += (F[6, d])

rule1 *= p
rule1

In [ ]:
rule2 = 0
for d in range(0, L):
    rule2 += (F[6, d])
rule2 *= p

for d in range(L, q):
    # print(d-L)
    rule2 += p*(F[6, d])*z[d-L]
rule2

In [ ]:
for d in range(0, q):
    print(round(F[6, d], 3))

In [ ]:
# d = 1
# n = 2
# p = len(actual_sales.loc[(actual_sales['item_nbr']==item_nbr) & (actual_sales['unit_sales']==n)]) / 365
# nbinom.cdf(k=d, n=n, p=p)

In [ ]:
# p = len(actual_sales.loc[(actual_sales['item_nbr']==item_nbr) & (actual_sales['unit_sales']==n)]) / 365
# rv = nbinom(n, p)
# fig, ax = plt.subplots(1, 1)
# x = np.arange(nbinom.ppf(0.01, n, p),
#               nbinom.ppf(0.99, n, p))
# ax.vlines(x, 0, rv.pmf(x), colors='k', linestyles='-', lw=1,
#         label='frozen pmf')
# ax.legend(loc='best', frameon=False)
# plt.show()

In [ ]:
# len(actual_sales.loc[(actual_sales['item_nbr']==item_nbr) & (actual_sales['unit_sales']==n)]) / 365

In [ ]:
# raw_df['month'] = raw_df['date'].apply(lambda d: datetime.datetime.strptime(d, '%Y-%m-%d').month)
# df = pd.get_dummies(raw_df, columns=['weekday', 'month', 'type'], drop_first=True)

# # 用2014年的資料當訓練集
# item_nbr = integer_items[0]
# from_date = '2014-01-01'
# to_date = '2014-12-31'
# test_day = 180


# ## poisson
# train_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>='2014-01-01') & (df['date']<='2014-12-31')]
# X_train = train_data.iloc[:, 8:]
# y_train = train_data[['unit_sales']]

# test_date = (datetime.datetime.strptime(to_date, "%Y-%m-%d") + datetime.timedelta(days=test_day)).strftime("%Y-%m-%d")
# test_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>to_date) & (df['date']<=test_date)]
# X_test = test_data.iloc[:, 8:]
# y_test = test_data[['unit_sales']]

# poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
# poisson_model.summary()

# poisson_pred = poisson_model.get_prediction(X_test)
# pred_summary_frame = poisson_pred.summary_frame()

# pred_counts = pred_summary_frame['mean']
# actual_counts = y_test['unit_sales']
# fig = plt.figure(figsize = (15, 8))
# predicted, = plt.plot(X_test.index, round(pred_counts), 'go-', label='Predicted counts')
# actual, = plt.plot(X_test.index, actual_counts, 'ro-', label='Actual counts')
# plt.legend(handles=[predicted, actual])
# plt.show()

In [ ]:
# ## NB2
# mu_df = pd.DataFrame()
# mu_df = pd.DataFrame(poisson_model.mu, columns=['mu'])
# mu_df['item_nbr'] = item_nbr
# mu_df = mu_df.reset_index(drop=True).reset_index()
# mu_df.rename(columns={'index':'day'}, inplace=True)

# # fit auxiliary OLS regression model to get alpha
# train_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=to_date)]
# train_data.reset_index(drop=True, inplace=True)
# train_data['LAMBDA'] = mu_df['mu'].loc[mu_df['item_nbr']==item_nbr]
# train_data['AUX_OLS_DEP'] = train_data.apply(lambda x: ((x['unit_sales'] - x['LAMBDA'])**2 - x['LAMBDA']) / x['LAMBDA'], axis=1)

# # alpha
# ols_expr = """AUX_OLS_DEP ~ LAMBDA - 1"""
# aux_olsr_model = smf.ols(ols_expr, train_data).fit()
# print('alpha:', aux_olsr_model.params[0])
# print('t-value:', aux_olsr_model.tvalues)

# train_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>=from_date) & (df['date']<=to_date)]
# X_train = train_data.iloc[:, 8:]
# y_train = train_data[['unit_sales']]

# nb2_model = sm.GLM(y_train, X_train,family=sm.families.NegativeBinomial(alpha=aux_olsr_model.params[0])).fit()
# print(nb2_model.summary())

# test_data = df.loc[(df['item_nbr']==item_nbr) & (df['date']>to_date) & (df['date']<=test_date)]
# X_test = test_data.iloc[:, 8:]
# y_test = test_data[['unit_sales']]

# nb2_pred_df = pd.DataFrame(round(nb2_model.predict(X_test)), columns=['pred'])
# nb2_pred_df['item_nbr'] = item_nbr
# nb2_pred_df = nb2_pred_df.reset_index(drop=True).reset_index()
# nb2_pred_df.rename(columns={'index':'day'}, inplace=True)

# nb2_pred = nb2_model.get_prediction(X_test)
# pred_summary_frame = nb2_pred.summary_frame()

# pred_counts = pred_summary_frame['mean']
# actual_counts = y_test['unit_sales']
# fig = plt.figure(figsize = (15, 8))
# predicted, = plt.plot(X_test.index, round(pred_counts), 'go-', label='Predicted counts')
# actual, = plt.plot(X_test.index, actual_counts, 'ro-', label='Actual counts')
# plt.legend(handles=[predicted, actual])
# plt.show()


In [ ]:
# def get_gp_alpha(N, k, p, item_nbr):
    
#     # 計算模型的alpha
#     alpha_df = gp_alpha_df.loc[gp_alpha_df['item_nbr']==item_nbr]
#     alpha = sum(((abs(alpha_df['true'] - alpha_df['pred']) / alpha_df['pred']**0.5) - 1) * alpha_df['pred']**(1-p)) / N-k-1

#     return alpha